In [ ]:
!pip install python-dotenv
!pip install jiwer
!pip install gradio
!pip install hopsworks

In [ ]:
# %load 3_feature_views_and_training_dataset.py
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store() 

In [5]:
air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality_fg',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg_clean',
    version = 1
)

In [6]:
query = air_quality_fg.select_all().join(weather_fg.select_all())
#query = air_quality_fg.select_all().join(weather_fg.select_all(include_event_time=False))
query_show = query.show(5)
query.read()

,date,pm25,tempmin,feelslikemin,feelslike,dew,humidity,precipcover,windgust,sealevelpressure,visibility
0,1673654400000,157,18.0,18.0,23.3,19.7,83.6,16.67,40.0,1008.8,11.4
1,1672444800000,70,6.9,4.9,10.8,5.4,66.9,0.00,36.4,1026.8,13.4
2,1672358400000,99,8.0,5.5,11.2,4.9,63.3,0.00,36.0,1027.1,13.2
3,1672272000000,133,10.9,10.9,13.8,4.5,53.8,0.00,35.2,1026.0,13.2
4,1672185600000,125,12.6,12.6,15.6,5.6,52.0,0.00,35.2,1023.6,13.3
...,...,...,...,...,...,...,...,...,...,...,...
361,1609804800000,122,15.1,15.1,18.3,3.3,37.5,0.00,25.0,1021.0,13.4
362,1609718400000,103,14.3,14.3,17.1,0.6,33.2,0.00,22.1,1022.0,13.5
363,1609632000000,121,5.8,4.7,13.8,0.0,42.1,0.00,31.6,1023.1,12.9
364,1609545600000,84,4.8,3.2,10.8,-4.1,39.1,0.00,25.2,1023.8,13.2


In [7]:
[t_func.name for t_func in fs.get_transformation_functions()]

['label_encoder', 'standard_scaler', 'min_max_scaler', 'robust_scaler']

In [8]:
col_names = query_show.columns
print(col_names)

Index(['date', 'pm25', 'tempmin', 'feelslikemin', 'feelslike', 'dew',
       'humidity', 'precipcover', 'windgust', 'sealevelpressure',
       'visibility'],
      dtype='object')


In [9]:
category_cols = ['date','pm25']

mapping_transformers = {col_name:fs.get_transformation_function(name='standard_scaler') for col_name in col_names if col_name not in category_cols}
category_cols = {col_name:fs.get_transformation_function(name='label_encoder') for col_name in category_cols if col_name not in ['date','pm25']}

mapping_transformers.update(category_cols)

In [11]:
feature_view = fs.create_feature_view(
    name = 'air_quality_fv',
    version = 1,
    labels=["pm25"],
    transformation_functions = mapping_transformers,
    query = query
)

feature_view = fs.get_feature_view(
    name = 'air_quality_fv',
    version = 1
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5465/fs/5372/fv/air_quality_fv/version/1
